In [2]:
from pydoc import cli
import random
from itertools import groupby,permutations,combinations
import logging
from types import NoneType
import networkx as nx

In [3]:
logging.basicConfig(level=logging.DEBUG)

In [4]:
def problem(N, seed=None):
    random.seed(seed)
    return [list(set(random.randint(0,N-1) for n in range(random.randint(N // 5, N // 2)))) for n in range(random.randint(N,N*5))]

In [5]:
def k_cliques(graph):
    # 2-cliques
    cliques = [{i, j} for i, j in graph.edges() if i != j]
    k = 2

    while cliques:
        # result
        yield k, cliques

        # merge k-cliques into (k+1)-cliques
        cliques_1 = set()
        for u, v in combinations(cliques, 2):
            w = u ^ v
            if len(w) == 2 and graph.has_edge(*w):
                cliques_1.add(tuple(u | w))

        # remove duplicates
        cliques = list(map(set, cliques_1))
        k += 1

In [6]:
def cliques(graph):
    # logging.debug(f"cliques for size {size_k}")
    clq = {}
    for k, cliques in k_cliques(graph):
        # if k == size_k:
        #     # print('%d-cliques = %d, %s.' % (k, len(cliques), cliques))
        #     return cliques
        # logging.debug(f"cliques of size {k}: {cliques}")
        clq[k]=cliques
    return clq

In [7]:
K = 5
sets = problem(K, 42)
sets = [list(s) for s in sets]
sets.sort()
sets = list(k for k,_ in groupby(sets))
sets_dict = {i: set(s) for i, s in enumerate(sets)}
graph = nx.Graph()
graph.add_nodes_from(range(len(sets)))

for z in permutations(sets_dict.items(), 2):
    if(len(z[0][1].intersection(z[1][1])) <= 2):
        graph.add_edge(z[0][0],z[1][0])
        graph.add_edge(z[1][0],z[0][0])

logging.info(f"sets_dict: {sets_dict}")

In [8]:
found = False
c = 1
best_w = K*K

partial_res = cliques(graph)

if partial_res is not None and not found:
    # logging.debug(f"partial_res: {partial_res.items()}")
    for k,c in partial_res.items():
        res_set = set()
        res_list = list()
        logging.info(f"k : {k}")
        logging.info(f"c : {c}")
        for s in c:
            for i in s:
                res_set = res_set.union(sets_dict[i])
                res_list.append(sets_dict[i])
            logging.debug(f"res_set: {res_set}")
            logging.debug(f"res_list: {res_list}")
            coverage = len(res_set)
            weight = sum(len(x) for x in res_list)
            if(coverage == K):
                if(weight < best_w):
                    best_w = weight
                    logging.info(f"new best weight : {best_w}")
                    logging.info(f"local best len: {sum(len(x) for x in res_list)} \nres : {res_list}")
                    if(best_w == K):
                        found = True
                        logging.info(f"new best weight : {best_w}")
                        logging.info(f"global best len: {sum(len(x) for x in res_list)} \nres : {res_list}")
                        break

INFO:root:k : 2


TypeError: unsupported format string passed to list.__format__